## Monte Carlo Simulation ##

### Calculating pi ###

In [3]:
import random
import numpy as np

def estimate_pi(num_points):
    points_inside_circle = 0

    for _ in range(num_points):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)

        distance = x**2 + y**2
        if distance <= 1:
            points_inside_circle += 1

    return 4 * points_inside_circle / num_points

# Repeat the simulation 5 times for each number of points and compute the mean
num_points_list = [1000, 10000, 100000, 1000000]
num_simulations = 5

for num_points in num_points_list:
    pi_estimates = []
    for _ in range(num_simulations):
        estimated_pi = estimate_pi(num_points)
        pi_estimates.append(estimated_pi)
    
    mean_pi = np.mean(pi_estimates)
    print(f"Number of points: {num_points}, Mean Estimated Pi: {mean_pi}")


Number of points: 1000, Mean Estimated Pi: 3.1184000000000003
Number of points: 10000, Mean Estimated Pi: 3.14272
Number of points: 100000, Mean Estimated Pi: 3.1382879999999997
Number of points: 1000000, Mean Estimated Pi: 3.1425736
